In [ ]:
import getpass
import os
import torch
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

#os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = config['API_KEYS']['huggingface_token']
#torch.cuda.empty_cache()

In [2]:
# Load pdf

from langchain.document_loaders import PyMuPDFLoader

########## To load a list of pdfs ##########

# from langchain_community.document_loaders import PyPDFDirectoryLoader

# loader=PyPDFDirectoryLoader("./us_census")

############################################

file_path = "C:/Users/Asus/Downloads/Informe_Tecnico_InstrumentoFortalecePYME.pdf"

loader = PyMuPDFLoader(file_path=file_path)

docs = loader.load()

print(len(docs))

36


In [3]:
# Make splits

from langchain_text_splitters import  RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

177


In [53]:
# Generate embeddings

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True}
    )

In [54]:
# Vector store

from langchain.vectorstores import FAISS
# Could try with Chroma

vector_store = FAISS.from_documents(documents=all_splits, embedding=embeddings)

ids = vector_store.add_documents(documents=all_splits)

In [103]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline


################################################### VOY POR EL 23:20 DEL VIDEO RAG DEL INDIO #########################################


# LLM Model to QA
#model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "EleutherAI/gpt-neo-1.3B"
#model_name = "t5-small"
model_name = "gpt2"

# Depending on task, objective can be one or other. Different models are better for one or other task
#objective = "text-generation"  # text-generation is more open-ended responses or more detailed anwsers based on the context
#objective = "question-answering" # question-answering extract anwsers from the provided context (a document for example). Factual.
#hf_pipeline = pipeline(objective, model=model_name, tokenizer=model_name, device=0) # Device=0 to GPU

from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(
    repo_id=model_name,
    model_kwargs={"temperature":0.4, "max_length":100}
)

#llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

def ask_question2(hf, question: str):
    #docs = vector_store.similarity_search(question)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':3})

    docs = retriever.get_relevant_documents(question)

    docs_content = "\n\n".join(doc.page_content for doc in docs)

    hf.invoke(question)

    prompt_template="""
    Contexto relevante:
    {context}

    Pregunta: 
    {question}

    Respuesta concisa y precisa:
    """

    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    retrievalQA = RetrievalQA.from_chain_type(
        llm=hf,
        chain_type="stuff",
        retriever=retriever,
        verbose=True,
        #return_source_document=True,
        chain_type_kwargs={"prompt":prompt}
    )

    #result = retrievalQA.invoke({"query": question})
    result = retrievalQA.run(question)
    return result['result']

In [110]:
question = "Dame una  lista de las 2 ideas más importantes del texto"
answer = ask_question2(hf=hf, question=question)

print(answer)

docs_content: 23 
 
 
 
Figura 1. Diagrama Modelo Appelfeller y Feldmann (2018).

23 
 
 
 
Figura 1. Diagrama Modelo Appelfeller y Feldmann (2018).

23 
 
 
Figura 2. Primera página informe de diagnóstico.  
 
5.2. Resumen ejecutivo 
 
En segundo lugar, el informe presenta un resumen ejecutivo que describe los alcances del 
programa Fortalece Pyme de la Región de Antofagasta. Luego, continúa con  una reflexión 
sobre la importancia de la digitalización de las empresas. Finalmente presenta una breve 
explicación sobre la evaluación de las dimensiones de la digitalización. En la Figura 3 muestra 
la segunda página del informe automatizado.

    Contexto relevante:
    23 
 
 
 
Figura 1. Diagrama Modelo Appelfeller y Feldmann (2018).

23 
 
 
 
Figura 1. Diagrama Modelo Appelfeller y Feldmann (2018).

23 
 
 
Figura 2. Primera página informe de diagnóstico.  
 
5.2. Resumen ejecutivo 
 
En segundo lugar, el informe presenta un resumen ejecutivo que describe los alcances del 
programa Fo

In [ ]:
question_2 = "¿De qué no habla el texto?"
answer2 = load_qa_chain(llm, chain_type="stuff", verbose=True)